# How to Perform a VASP calculation
## Input Files

Find crystallographic information file (CIF). Best website is https://www.ccdc.cam.ac.uk/structures/?

Now we need to convert CIF file to a usable input file.

cif2cell [PATH]/'CIF'.cif -p vasp --setup-all
    
This produces 4 files: INCAR, POSCAR, POTCAR AND KPOINTS.

#### INCAR

The INCAR produced by cif2cell will be missing some parameters and some will be different from what we want. An example INCAR for a geometry optimisation looks like this (some parameters are simply for speed and will not affect accuracy, the most important ones are commented):
    
ENCUT   = 600      # Cut off Energy

PREC    = Accurate # Sets precision - related to ENCUT

LREAL   = .FALSE.  # Real or reciprocal space

ISMEAR  = -5       # Sets orbital partial occupancy

SIGMA   = 0.05     # Sets width in eV of smearing

IBRION  = 1        # Sets method of moving ions.

NFREE   = 10       

POTIM   = 0.2      # Step widths

ISIF    = 3        # Optimise ions, cell dimensions or both

NSW     = 1000     # Number of iterations before finishing job (1000 selected so it never finishes before 48 hours)

NELMIN  = 4        # Min number of SCF cycles for each set of ionic positions 

LPLANE  = .TRUE.   

NCORE   = 8

NSIM    = 2 

ALGO    = Fast     # Determines what algorithm is used

EDIFF   = 5.0E-6   # Determines accuracy at which program stops

LMAXMIX = 4

I recommend looking at vasp.at/wiki/index.php/Category:INCAR for in-depth information about each tag and comprehensive list of all possible tags.

#### KPOINTS

The KPOINTS file will look like:

Generated by cif2...

0

Gamma

X Y Z

0 0 0

Where X Y Z can be equal to eachother but also can be different numbers. I would recommend trying at the given KPOINT values but I have run into problems when using odd and even KPOINTS. Try to use all odd or all even KPOINTS.

#### POTCAR AND POSCAR

These files are correct and should be left as is.

## Convergence

To ensure high accuracy, which is very important for phonon calculations, the parameters must be converged. The two parameters in VASP to be converged are KPOINTS and ENCUT. 

IBRION should be set to -1 (and NSW = 0) to perform a single-point energy calculation, which will save time.

#### KPOINTS

Start with the recommended KPOINT values and a low value for ENCUT such as 300.

Increase KPOINT values by 1 each run until it the final energy does not change within a margin of significance (usually 1 or 0.1%). Whilst varying KPOINTS keep ENCUT fixed.

#### ENCUT

Start with recommended KPOINTS and fix them for these calculations. Start at recommended ENCUT, which is found in the generated INCAR. Increase ENCUT by 50 or 100 each time until, as above, the energy of the system is not changing within 1% or 0.1%.

To run a job, the command is: runvasp [Jobname] [Number of Processesors] [Hours]

e.g. runvasp LiLeucine 24 48
    
Max runtime is 48 hours.

## Geometry Optimisation

Once you have your converged parameters, we can now start a geometry optimisation (GO). The parameters are the same as in the example INCAR above.

If the job finishes within 48 hours, we can move onto the next step. However, if it didn't, we need to keep going.

To check if a GO has finished, use the command: grep accuracy OUTCAR

If finished, this will output: reached required accuracy ....

If not, this will output nothing.

To continue a job, make a new directory and copy over the KPOINTS, POTCAR, INCAR and WAVECAR files. You also need to copy over the CONTCAR file and rename it to POSCAR. 

Additionally, two new tags must be added to the INCAR:

ISTART = 1

ICHARG = 0

These enable the calculation to continue where it left off.

Once this has completed (or if it doesn't; parameters in the INCAR file must be experimented with), a more precise GO is performed whilst fixing the unit cell dimensions. This results in a highly accurate structure with which we can calculate the system's phonon modes.

This is easily done by using: vasp_fixedcell [Target Dir] [Proc] [Time]

The same procedure applies for checking if finished and continuing the job if not.

## Calculating Phonon Frequencies and Intensities

Now that we have a highly optimised system, the phonon modes can be calculated. This is done using the command:

vasp_dielectric [Target Dir] [Proc] [Time] 

After 48 hours, using the command: grep THz OUTCAR

will determine whether the calculation has finished. If there is no output, the calculation needs longer than 48 hours (in VASP) and we must use Phonopy (See Phonopy Notebook) to calculate the phonon modes.

If there is an output, the calculation finished successfully and can now be analysed in PDielec (See PDielec Notebook).